In [12]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import pandas as pd
from pathlib import Path
import colorsys
import math
# print(pd.__version__)

In [13]:

path_to_data :Path = Path("./../data")
data_seq= path_to_data.joinpath("mandelbrot_amd_seq_.csv")
data_openmp= path_to_data.joinpath("mandelbrot_amd_openmp_.csv")
data_cuda= path_to_data.joinpath("mandelbrot_cuda_cuda_.csv")
data_mpi= path_to_data.joinpath("mandelbrot_mpi.csv")
for x in [data_seq, data_openmp, data_cuda, data_mpi]:
	print(x)
	assert x.exists()
	assert x.is_file()
	assert x.suffix == ".csv"
df_openmp : pd.DataFrame = pd.read_csv(data_openmp)
df_cuda : pd.DataFrame = pd.read_csv(data_cuda)
df_mpi : pd.DataFrame = pd.read_csv(data_mpi)
df_seq : pd.DataFrame = pd.read_csv(data_seq)

# Add implementation column
df_seq['Implementation'] = 'Seq'
df_openmp['Implementation'] = 'OpenMP'
df_cuda['Implementation'] = 'CUDA'
df_mpi['Implementation'] = 'MPI'


..\data\mandelbrot_amd_seq_.csv
..\data\mandelbrot_amd_openmp_.csv
..\data\mandelbrot_cuda_cuda_.csv
..\data\mandelbrot_mpi.csv


### Seq exec heatmap

In [14]:
# Filter for Sequential implementation
df_seq_temp = df_seq[['Resolution', 'Iterations', 'Time (seconds)']].copy()
# Pivot the dataframe
melted_df = df_seq_temp[["Resolution", "Iterations"]].astype(str)
melted_df["Time (seconds)"] = df_seq_temp["Time (seconds)"]
heatmap_df_seq = melted_df.pivot(index='Resolution', columns='Iterations', values='Time (seconds)')
# melted_df = pd.melt(heatmap_df, id_vars=['Resolution', 'Iterations'], value_vars=['Time (seconds)'],
#                     var_name='Metric', value_name='Time (seconds)')
# Create heatmap with Plotly
fig = px.imshow(
    heatmap_df_seq,
    labels=dict(x="Iterations", y="Resolution", color="Time (seconds)"),
    title="Sequential Execution Time Heatmap",
    aspect="auto",
    text_auto=True,
)

fig.show()

In [15]:
# Filter for Sequential implementation
thread_1_df = df_openmp[df_openmp['Threads'] == 1].copy()
# Pivot the dataframe
melted_df = thread_1_df[["Resolution", "Iterations"]].astype(str)
melted_df["Time (seconds)"] = thread_1_df["Time (seconds)"]
heatmap_df_openmp = melted_df.pivot(index='Resolution', columns='Iterations', values='Time (seconds)')

# Create heatmap with Plotly
fig = px.imshow(
    heatmap_df_openmp,
    labels=dict(x="Iterations", y="Resolution", color="Time (seconds)"),
    title="OpenMP 1 thread Execution Time Heatmap",
    aspect="auto",
    text_auto=True,
)
fig.update_xaxes(side="top")
fig.show()

In [16]:
time_openmp_seq_diff = df_seq_temp["Time (seconds)"] - thread_1_df["Time (seconds)"]
df_diff_seq_openmp = df_seq_temp.copy()
df_diff_seq_openmp["Time (seconds)"] = time_openmp_seq_diff
melted_df = df_diff_seq_openmp[["Resolution", "Iterations"]].astype(str)
melted_df["Time (seconds)"] = df_diff_seq_openmp["Time (seconds)"]
heatmap_df_openmp_diff = melted_df.pivot(index='Resolution', columns='Iterations', values='Time (seconds)')

fig = px.imshow(
    heatmap_df_openmp_diff,
    labels=dict(x="Iterations", y="Resolution", color="Time (seconds)"),
    title="Sequential time minus OpenMP 1 thread Execution Time Heatmap",
    aspect="auto",
    text_auto=True,
    # move x label to top
)
fig.update_xaxes(side="top")

fig.show()

### Best OpenMP scheduling Solution

In order to select the bestsolution on some basis we limit the number of iterations to 4000 and resolution to 8000 since this is the most computationally expensive solution. We will run the code on the given input and measure the time taken by the code to execute. The code which takes the least time will be selected

In [17]:
# Combine all DataFrames
types_of_scheduling_openmp = ["DYNAMIC", "STATIC", "GUIDED", "RUNTIME"]

# 1. Filter for OpenMP implementation
df_openmp_temp = df_openmp[['Threads', 'Resolution', 'Iterations', 'Time (seconds)', 'Scheduling']].copy()
df_openmp_temp = df_openmp_temp[df_openmp_temp['Threads'] != 1]
#Convert Resolution and Iterations as string
df_openmp_temp['Resolution'] = df_openmp_temp['Resolution'].astype(str)
df_openmp_temp['Iterations'] = df_openmp_temp['Iterations'].astype(str)

# Convert Resolution and Iterations in df_seq_temp to string
df_seq_temp['Resolution'] = df_seq_temp['Resolution'].astype(str)
df_seq_temp['Iterations'] = df_seq_temp['Iterations'].astype(str)

# Divide the dataframe into 4 based on the type of scheduling
df_openmp_dict = {}
for i in types_of_scheduling_openmp:
    df_openmp_dict[i] = df_openmp_temp[df_openmp_temp['Scheduling'] == i].copy()
#  Calculate the speedup for every iteration and resulution value
# Iterate through each scheduling type
for sched in types_of_scheduling_openmp:
    # Merge OpenMP dataframe with sequential dataframe on Resolution and Iterations
    merged_df = pd.merge(
        df_openmp_dict[sched],
        df_seq_temp[['Resolution', 'Iterations', 'Time (seconds)']],
        on=['Resolution', 'Iterations'],
        suffixes=('_openmp', '_seq')
    , how="inner", validate="many_to_many")
    
    # Calculate Speedup
    merged_df['Speedup'] = merged_df['Time (seconds)_seq'] / merged_df['Time (seconds)_openmp']
    
    # Calculate Efficiency
    merged_df['Efficiency'] = merged_df['Speedup'] / merged_df['Threads']
    
    # Update the dictionary with the new dataframe
    df_openmp_dict[sched] = merged_df

#Joining all openmp dataframes
df_openmp_all = pd.concat(df_openmp_dict.values(), ignore_index=True)
# Remove all records of resolution different than 8000 and iterations different than 4000
df_openmp_all_max_res_iter = df_openmp_all[(df_openmp_all['Resolution'] == '8000') & (df_openmp_all['Iterations'] == '4000')]

# 5. Create the line plot with enhanced structure
fig = px.line(
    df_openmp_all_max_res_iter,
    x='Threads',
    y='Time (seconds)_openmp',
    color='Scheduling',
    markers=True,
    title='Execution Time: OpenMP scheduling types - 8000x8000 resolution, 4000 iterations',
    labels={
        'Iterations': 'Number of Iterations',
        'Time (seconds)': 'Execution Time (seconds)',
        'Implementation': 'Implementation'
    },
        width=600
    
)

# Set x-axis to display only the actual thread values
fig.update_xaxes(type='category')

# Change marker symbols to crosses and increase size for better visibility
fig.update_traces(marker=dict(symbol='cross', size=10, line=dict(width=1, color='Black')))
fig.update_layout(
    legend=dict(
        x=0.69,
        y=0.99,
        bgcolor='rgba(255,255,255,0.5)',
        bordercolor='Black',
        borderwidth=1
    )
)
# Define ideal speedup based on threads
ideal_threads = [2, 4, 8, 16]
base_time = df_openmp_all_max_res_iter[df_openmp_all_max_res_iter['Threads'] == 2]['Time (seconds)_openmp'].values[0]
ideal_times = [base_time / (t / 2) for t in ideal_threads]

# Add ideal speedup line
fig.add_trace(
    go.Scatter(
        x=ideal_threads,
        y=ideal_times,
        mode='lines+markers',
        name='Ideal Speedup',
        line=dict(dash='dot', color='Red'),
        marker=dict(symbol='cross-thin', size=10)
    )
)
fig.show()

best_openmp_speedup = df_openmp_all_max_res_iter[df_openmp_all_max_res_iter["Time (seconds)_openmp"] == min(df_openmp_all_max_res_iter["Time (seconds)_openmp"])]

print(f"The best time exectuion is for: {best_openmp_speedup["Scheduling"].values} at {best_openmp_speedup["Time (seconds)_openmp"].values} therefore it will be used for calculating speedup")

The best time exectuion is for: ['DYNAMIC'] at [110.401] therefore it will be used for calculating speedup


### Calculating speedup (Base time is seq execuction)

In [18]:
# Get sequential times in pivot format
seq_times = df_seq.pivot(
    index=['Resolution', 'Iterations'],
    columns='Implementation',
    values='Time (seconds)'
)
seq_times = seq_times.rename(columns={'Seq': 'Sequential'})
seq_times = seq_times['Sequential']
# Combine with OpenMP data
best_open_mp_df = df_openmp_all[df_openmp_all["Scheduling"] == best_openmp_speedup["Scheduling"].values[0]].copy()
best_open_mp_df['Resolution'] = best_open_mp_df['Resolution'].astype(int)
best_open_mp_df['Iterations'] = best_open_mp_df['Iterations'].astype(int)
best_open_mp_df_pivoted = best_open_mp_df.pivot(
    index=['Resolution', 'Iterations'],
    columns='Threads',
    values='Time (seconds)_openmp'
)
speedup_df = best_open_mp_df_pivoted.copy()
best_open_mp_df_pivoted.columns.name = 'Threads'
# display(best_open_mp_df_pivoted)

# Calculate speedup (sequential time divided by parallel time)
for col in speedup_df.columns:
    speedup_df[col] = seq_times / best_open_mp_df_pivoted[col]
speedup_df.columns = [f'{n} threads (speedup)' for n in speedup_df.columns]
speedup_df.columns.name = 'Threads'
# print("speedup_df")
# display(speedup_df)
mismatch = ~seq_times.index.isin(best_open_mp_df_pivoted.index)
if mismatch.any():
    print("Mismatched indices:", seq_times.index[mismatch])
# Combine times and speedup
combined_results = pd.concat([
    pd.DataFrame(seq_times, columns=['Sequential']), 
    best_open_mp_df_pivoted, 
    speedup_df
], axis=1)
# Change column names so that threads , sequential to indicate that is it is seconds
combined_results.columns = [f'{c} (s)' if c == 'Sequential' else c for c in combined_results.columns]

display(combined_results)


Sequential (s)          2          4           8  \
Resolution Iterations                                                     
1000       1000               5.49536    2.84885    1.50649    0.857005   
           2000              10.83290    5.71449    2.93833    1.550570   
           4000              21.18800   11.29450    5.83943    2.995310   
2000       1000              21.67650   11.46120    6.07812    3.022910   
           2000              42.66820   22.62220   11.81360    6.063940   
           4000              85.08150   44.71630   23.32920   12.457200   
4000       1000              87.42960   45.69010   24.13100   12.295200   
           2000             172.43700   90.38070   47.14130   24.619700   
           4000             340.39100  178.86700   92.91790   48.755700   
8000       1000             348.09400  184.42700   96.18190   50.855300   
           2000             699.59800  360.68900  188.89100   97.645200   
           4000            1382.91000  716.25900  370.69100  194.125000   

                               16  2 threads (speedup)  4 threads (speedup)  \
Resolution Iterations                                                         
1000       1000          0.507952             1.928975             3.647791   
           2000          0.965742             1.895690             3.686754   
           4000          1.815290             1.875957             3.628436   
2000       1000          1.891570             1.891294             3.566317   
           2000          3.514120             1.886121             3.611786   
           4000          6.901590             1.902695             3.646996   
4000       1000          7.197010             1.913535             3.623124   
           2000         13.798900             1.907896             3.657875   
           4000         27.327300             1.903040             3.663352   
8000       1000         28.572600             1.887435             3.619122   
           2000         56.340400             1.939616             3.703713   
           4000        110.401000             1.930740             3.730627   

                       8 threads (speedup)  16 threads (speedup)  
Resolution Iterations                                             
1000       1000                   6.412285             10.818660  
           2000                   6.986399             11.217178  
           4000                   7.073725             11.671964  
2000       1000                   7.170739             11.459528  
           2000                   7.036382             12.141930  
           4000                   6.829906             12.327811  
4000       1000                   7.110873             12.148045  
           2000                   7.004025             12.496431  
           4000                   6.981563             12.456079  
8000       1000                   6.844793             12.182791  
           2000                   7.164694             12.417342  
           4000                   7.123812             12.526245

In [19]:
# 1. Reset the index to have 'Resolution' and 'Iterations' as columns
df_temp = combined_results.reset_index()
# 2. Identify speedup columns
speedup_cols = [col for col in list(df_temp.columns) if type(col) == str and 'threads (speedup)' in col]
# 3. Melt the DataFrame to long format for speedup data
df_melted = df_temp.melt(
    id_vars=['Resolution', 'Iterations'],
    value_vars=speedup_cols,
    var_name='Threads',
    value_name='Speedup'
)

# 4. Extract thread count from the 'Threads' column
df_melted['Thread Count'] = df_melted['Threads'].str.extract(r'(\d+)').astype(int)

# 5. Create a 'Scenario' column combining Resolution and Iterations
df_melted['Scenario'] = 'Res=' + df_melted['Resolution'].astype(str) + ', It=' + df_melted['Iterations'].astype(str)

# 6. Plot using Plotly Express
fig = px.line(
    df_melted,
    x='Thread Count',
    y='Speedup',
    color='Scenario',
    markers=True,
    title='Speedup vs Number of Threads',
    labels={
        'Thread Count': 'Number of Threads',
        'Speedup': 'Speedup (Sequential / Parallel)',
        'Scenario': 'Scenario (Resolution & Iterations)'
    }
)

# 7. Enhance the layout for better readability
fig.update_layout(
    xaxis=dict(
		type='category',  # Treat x-axis as categorical
        title='Number of Threads'),
    yaxis=dict(title='Speedup (Sequential / Parallel)'),
    legend_title_text='Scenario'
)

# 8. Display the plot
fig.show()

In [20]:
# 1. Ensure 'Thread Count' is treated as an ordered categorical variable
df_melted['Thread Count'] = pd.Categorical(
    df_melted['Thread Count'],
    categories=sorted(df_melted['Thread Count'].unique()),
    ordered=True
)

# 2. Identify the maximum speedup for each thread count
max_speedups = df_melted.loc[df_melted.groupby('Thread Count')['Speedup'].idxmax()]

# 3. Combine multiple qualitative color palettes for increased variety
combined_palette = (
    px.colors.qualitative.Plotly +
    px.colors.qualitative.D3 +
    px.colors.qualitative.Set1 +
    px.colors.qualitative.Set3 +
    px.colors.qualitative.Dark2 +
    px.colors.qualitative.Pastel1
)
scenarios = df_melted['Scenario'].unique()
if len(scenarios) > len(combined_palette):
    raise ValueError(f"Number of scenarios ({len(scenarios)}) exceeds the number of available colors ({len(combined_palette)}). Consider generating more colors programmatically.")

color_discrete_map = {scenario: combined_palette[i] for i, scenario in enumerate(scenarios)}

# 4. Assign colors to max_speedups based on their Scenario
max_speedups_colors = max_speedups['Scenario'].map(color_discrete_map).tolist()

# 5. Create the bar plot with the extended color mapping
fig_bar = px.bar(
    df_melted,
    x='Thread Count',
    y='Speedup',
    color='Scenario',
    barmode='group',  # Groups bars side by side for each thread count
    title='Speedup vs Number of Threads',
    labels={
        'Thread Count': 'Number of Threads',
        'Speedup': 'Speedup (Sequential / Parallel)',
        'Scenario': 'Scenario (Resolution & Iterations)'
    },
    text='Speedup',  # Optional: Show speedup values on bars
    color_discrete_map=color_discrete_map
)

# 6. Calculate an offset for the markers to position them above the bars
offset = df_melted['Speedup'].max() * 0.15  # 15% of the maximum speedup

# 7. Add "Max" markers with scenario names on top of maximum bars
fig_bar.add_trace(
    go.Scatter(
        x=max_speedups['Thread Count'],
        y=max_speedups['Speedup'] + offset,  # Position above the bar
        mode='markers+text',
        marker=dict(
            size=12,
            color=max_speedups_colors,  # Use the same color as the corresponding bar
            symbol='diamond'
        ),
        # Combine scenario name and speedup in the text
        text=max_speedups['Speedup'].round(4).astype(str),
        textposition="top center",
        showlegend=False
    )
)

# 8. Enhance the layout for better readability
fig_bar.update_layout(
	width=1000,    # Set the desired width
    height=600,   # Set the desired height
    xaxis_title='Number of Threads',
    yaxis_title='Speedup (Sequential / Parallel)',
    legend_title='Scenario',
    xaxis=dict(type='category'),  # Ensure x-axis is treated as categorical
    template='plotly_white',      # Optional: a clean white background
    title={
        'text': "Speedup per thread configuration",
        'y': 0.95,
        'x': 0.5,
        'xanchor': 'auto',
        'yanchor': 'auto'
    },
    margin=dict(t=100, l=50, r=50, b=50),  # Adjust margins as needed
)

# 9. Optionally, adjust the text on the bars for better visibility
fig_bar.update_traces(texttemplate='%{text:.3f}')

# 10. Adjust y-axis range to ensure annotations are visible
fig_bar.update_yaxes(range=[0, df_melted['Speedup'].max() * 1.3])
fig_bar.update_xaxes(tickvals=df_melted['Thread Count'].unique())

# 11. Display the bar plot
fig_bar.show()


C:\Users\szink\AppData\Local\Temp\ipykernel_18932\440099886.py:9: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



## MPI

### MPI results are not 1:1 processor that is being used for SEQ and OMP. So we will calculate the speedup for the best OMP solution and compare it with the MPI solution
Nodes used in full:
hpcocapie01
hpcocapie03
hpcocapie04
hpcocapie05
hpcocapie06
hpcocapie07
hpcocapie08


### Table for MPI

In [21]:
df_mpi_temp = df_mpi[['Resolution', 'Iterations', 'Time (seconds)', 'Cores']].copy()
display(df_mpi_temp)

,Resolution,Iterations,Time (seconds),Cores
0,1000,1000,4.371690,16
1,1000,2000,9.078950,16
2,1000,4000,18.009600,16
3,2000,1000,18.314700,16
4,2000,2000,36.121100,16
...,...,...,...,...
69,1000,4000,0.459665,512
70,2000,1000,1.879520,512
71,2000,2000,1.434030,512
72,2000,4000,3.472470,512


### plot for MPI

In [22]:
combined_df = pd.concat([df_seq, best_open_mp_df, df_cuda, df_mpi], ignore_index=True)
# print(combined_df['Implementation'].unique())

# Pivot the DataFrame to have implementations as columns
pivot_df = combined_df.pivot_table(
    index=['Iterations', 'Resolution'],
    columns='Implementation',
    values='Time (seconds)'
).reset_index()


# for x in pivot_df.keys():
# 	print(x)
# Calculate Speedup
pivot_df['Speedup_CUDA'] = pivot_df['Seq'] / pivot_df['CUDA']
pivot_df['Speedup_MPI'] = pivot_df['Seq'] / pivot_df['MPI']
pivot_df['Speedup_OpenMP'] = pivot_df['Seq'] / pivot_df['OpenMP']
display(pivot_df.head())

# Display Speedup Results
# print(pivot_df[['Iterations', 'Resolution', 'Speedup_OpenMP', 'Speedup_CUDA', 'Speedup_MPI']])

KeyError: 'OpenMP'